## using Adaboost classifier we will prepare a model on fraud data treating those who have taxable_income <= 30000 as "Risky" and others are "Good"

In [40]:
#libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split 

#adaboost library
from sklearn.ensemble import AdaBoostClassifier

In [18]:
df = pd.read_csv("G:\Github\DS-assignments-python\Adaboost algorithm\Fraud_check.csv")

In [19]:
df.head(10)

,Undergrad,Marital.Status,Taxable.Income,City.Population,Work.Experience,Urban
0,NO,Single,68833,50047,10,YES
1,YES,Divorced,33700,134075,18,YES
2,NO,Married,36925,160205,30,YES
3,YES,Single,50190,193264,15,YES
4,NO,Married,81002,27533,28,NO
5,NO,Divorced,33329,116382,0,NO
6,NO,Divorced,83357,80890,8,YES
7,YES,Single,62774,131253,3,YES
8,NO,Single,83519,102481,12,YES
9,YES,Divorced,98152,155482,4,YES


In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 600 entries, 0 to 599
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Undergrad        600 non-null    object
 1   Marital.Status   600 non-null    object
 2   Taxable.Income   600 non-null    int64 
 3   City.Population  600 non-null    int64 
 4   Work.Experience  600 non-null    int64 
 5   Urban            600 non-null    object
dtypes: int64(3), object(3)
memory usage: 28.2+ KB


In [21]:
df.nunique()

Undergrad            2
Marital.Status       3
Taxable.Income     599
City.Population    598
Work.Experience     31
Urban                2
dtype: int64

In [22]:
df1 = df

In [23]:
df1["Taxable.Income"] = pd.Series(np.where(df["Taxable.Income"]<= 30000 ,1,0))

In [28]:
df1.rename(columns = {"Taxable.Income":"Tax_Inc_Risky"}, inplace = True)

Treating 1 as Risky and 0 as Good

In [29]:
df1.head(10)

,Undergrad,Marital.Status,Tax_Inc_Risky,City.Population,Work.Experience,Urban
0,NO,Single,0,50047,10,YES
1,YES,Divorced,0,134075,18,YES
2,NO,Married,0,160205,30,YES
3,YES,Single,0,193264,15,YES
4,NO,Married,0,27533,28,NO
5,NO,Divorced,0,116382,0,NO
6,NO,Divorced,0,80890,8,YES
7,YES,Single,0,131253,3,YES
8,NO,Single,0,102481,12,YES
9,YES,Divorced,0,155482,4,YES


In [31]:
df["Tax_Inc_Risky"].value_counts()

0    476
1    124
Name: Tax_Inc_Risky, dtype: int64

In [26]:
124/600

0.20666666666666667

In [33]:
#Creating dataframes with all numerical and dummies
dfd = pd.get_dummies(data = df , columns = ['Undergrad','Marital.Status','Urban'], drop_first = True)

In [34]:
dfd.head()

,Tax_Inc_Risky,City.Population,Work.Experience,Undergrad_YES,Marital.Status_Married,Marital.Status_Single,Urban_YES
0,0,50047,10,0,0,1,1
1,0,134075,18,1,0,0,1
2,0,160205,30,0,1,0,1
3,0,193264,15,1,0,1,1
4,0,27533,28,0,1,0,0


#### so the data is nicely unbalance..
now adaboost is a algorithm which counters the imbalanced data  by giving more weight to datapoints which are not classified correctly... in this case minority class datapoints
Lets see how it performs

In [38]:
dfd_x = dfd.iloc[:,1:]
dfd_y = dfd.iloc[:,0]

In [41]:
dfd_tr_x,dfd_ts_x,dfd_tr_y,dfd_ts_y = train_test_split(dfd_x,dfd_y, test_size = 0.25,shuffle = True , random_state = 13)

AdaBoostClassifier(base_estimator=None, *, n_estimators=50, learning_rate=1.0, algorithm='SAMME.R', random_state=None)

- base_estimator - model that u wnat to have as classifier , default is decision tree
- n_estimator - number of stumps/tress / classifiaction learners you want to build
- learning rate - Weight applied to each classifier at each boosting iteration. A higher learning rate increases the contribution of each classifier. There is a trade-off between the learning_rate and n_estimators parameters.


In [95]:
#Creating a adaboost model
mark1 = AdaBoostClassifier()

In [96]:
mark1.fit(dfd_tr_x,dfd_tr_y)

AdaBoostClassifier()

In [97]:
pred_tr = mark1.predict(dfd_tr_x)
pred_ts = mark1.predict(dfd_ts_x)

In [98]:
np.mean(pred_tr == dfd_tr_y)

0.7822222222222223

In [99]:
np.mean(pred_ts == dfd_ts_y)

0.84

In [100]:
pd.crosstab(dfd_tr_y,pred_tr )

col_0,0,1
Tax_Inc_Risky,,
0,348,2
1,96,4


In [101]:
pd.crosstab(dfd_ts_y,pred_ts)

col_0,0,1
Tax_Inc_Risky,,
0,125,1
1,23,1
